In [0]:
from pyspark.sql.functions import (
    col, trim, regexp_replace, initcap, when
)

# 1. Ensure schema exists
spark.sql("CREATE DATABASE IF NOT EXISTS lakehouse.silver")

# 2. Read Bronze table
bronze_df = spark.table("lakehouse.bronze.crm_cust_info")

# 3. Clean data (equivalent to DELETE WHERE cst_id IS NULL)
bronze_clean_df = bronze_df.filter(col("cst_id").isNotNull())

# 4. Transformations
silver_df = (
    bronze_clean_df
    .withColumn("cst_id", col("cst_id").cast("int"))
    .withColumn("cst_key", col("cst_key").cast("string"))
    .withColumn(
        "cst_firstname",
        initcap(regexp_replace(trim(col("cst_firstname")), "\\s+", " "))
    )
    .withColumn(
        "cst_lastname",
        initcap(regexp_replace(trim(col("cst_lastname")), "\\s+", " "))
    )
    .withColumn(
        "cst_marital_status",
        when(trim(col("cst_marital_status")) == "S", "Single")
        .when(trim(col("cst_marital_status")) == "M", "Married")
        .otherwise(col("cst_marital_status"))
    )
    .withColumn(
        "cst_gndr",
        when(trim(col("cst_gndr")) == "F", "Female")
        .when(trim(col("cst_gndr")) == "M", "Male")
        .otherwise(col("cst_gndr"))
    )
    .withColumn("cst_create_date", col("cst_create_date").cast("date"))
)

# 5. Write to Silver layer
(
    silver_df
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("lakehouse.silver.crm_cust_info")
)

print("Silver CRM customer info created successfully")
